<a href="https://colab.research.google.com/github/jaysalomon/Bio-Spin/blob/main/Thermal_Stability_Simulation_(Approximate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Assume Magnetosome and Simulation classes are defined here ---
# <<< PASTE the Magnetosome and Simulation class definitions here >>>
# --- Physical Constants ---
GAMMA_LL = 2.2128e5
ALPHA = 0.02
MS = 4.8e5
KB = 1.380649e-23
T_TEMP = 300 # Temperature (K) - NOW USED for thermal field
MU0 = 4 * np.pi * 1e-7

class Magnetosome:
    """ Represents a single magnetosome nanoparticle. (Copied from magnetosome_sim_env) """
    def __init__(self, position, initial_M_direction, volume, Ku, easy_axis=[0, 0, 1]):
        self.pos = np.array(position, dtype=float)
        initial_M_direction = np.array(initial_M_direction, dtype=float)
        norm = np.linalg.norm(initial_M_direction)
        if norm < 1e-9: raise ValueError("Initial magnetization direction cannot be a zero vector.")
        self.M = (initial_M_direction / norm) * MS
        self.V = float(volume)
        self.Ku = float(Ku)
        self.Ms = MS
        self.easy_axis = np.array(easy_axis, dtype=float)
        norm_ea = np.linalg.norm(self.easy_axis)
        if norm_ea < 1e-9: raise ValueError("Easy axis cannot be a zero vector.")
        self.easy_axis /= norm_ea

    def anisotropy_field(self):
        M_norm = np.linalg.norm(self.M)
        if M_norm < 1e-9: return np.zeros(3)
        M_unit = self.M / M_norm
        M_physical = M_unit * self.Ms
        M_dot_easy_axis = np.dot(M_physical, self.easy_axis)
        H_anis_magnitude = (2 * self.Ku) / (MU0 * self.Ms)
        H_anis = H_anis_magnitude * M_dot_easy_axis / self.Ms * self.easy_axis
        return H_anis

    def normalize_M(self):
         norm_M = np.linalg.norm(self.M)
         if norm_M > 1e-9: self.M = (self.M / norm_M) * self.Ms

class Simulation:
    """ Manages the simulation environment. (Modified for Thermal Field) """
    def __init__(self, magnetosomes, H_ext=[0, 0, 0], temperature=T_TEMP, approx_dt=1e-12):
        """
        Initializes the Simulation instance.

        Args:
            magnetosomes (list): List of Magnetosome objects.
            H_ext (list/tuple/np.array): External magnetic field.
            temperature (float): Temperature in Kelvin for thermal field calculation.
            approx_dt (float): Approximate time step used for scaling thermal noise.
                               THIS IS AN APPROXIMATION for use with solve_ivp.
        """
        self.magnetosomes = magnetosomes
        self.H_ext = np.array(H_ext, dtype=float)
        self.temperature = temperature
        self.approx_dt = approx_dt # Store the approximate dt

    def calculate_dipolar_field(self, target_index, current_M_vectors):
        # (Identical to previous version - code omitted for brevity)
        H_dip = np.zeros(3)
        target_magnetosome = self.magnetosomes[target_index]
        for i, other_magnetosome in enumerate(self.magnetosomes):
            if i == target_index: continue
            r_vec = target_magnetosome.pos - other_magnetosome.pos
            r_dist = np.linalg.norm(r_vec)
            if r_dist < 1e-12: continue
            r_hat = r_vec / r_dist
            M_j = current_M_vectors[i]
            m_j = M_j * other_magnetosome.V
            term1 = 3 * np.dot(m_j, r_hat) * r_hat
            term2 = m_j
            H_dip += (1 / (4 * np.pi * r_dist**3)) * (term1 - term2)
        return H_dip


    def calculate_thermal_field(self, magnetosome):
        """
        Calculates the stochastic thermal field H_th for a given magnetosome.
        The field components are drawn from a Gaussian distribution.
        Magnitude is scaled approximation based on fluctuation-dissipation theorem.
        H_th ~ sqrt(2 * alpha * kB * T / (gamma_LL * mu0 * Ms * V * dt)) * Gaussian(0,1)

        Args:
            magnetosome (Magnetosome): The magnetosome object.

        Returns:
            np.array: The 3D stochastic thermal field vector (A/m).
        """
        if self.temperature <= 0 or self.approx_dt <= 0:
            return np.zeros(3)

        # Strength of the thermal fluctuations
        # Note: mu0 is needed here if using gamma_LL (m/As). If using gamma' = mu0*gamma_LL, mu0 is omitted.
        # Let's use the form consistent with H_eff being in A/m and using GAMMA_LL.
        thermal_strength_factor = np.sqrt(
            (2 * ALPHA * KB * self.temperature) /
            (GAMMA_LL * MU0 * magnetosome.Ms * magnetosome.V * self.approx_dt)
        )

        # Generate random Gaussian numbers for each component
        random_gaussian = np.random.normal(0.0, 1.0, 3)

        H_th = thermal_strength_factor * random_gaussian
        return H_th

    def calculate_Heff(self, index, current_M_vectors):
        """
        Calculates the total effective magnetic field including thermal field.
        Heff = Hext + Hanis + Hdip + Hth
        """
        magnetosome = self.magnetosomes[index]

        # Update the magnetosome's M vector temporarily for anisotropy calculation
        original_M = magnetosome.M.copy()
        magnetosome.M = current_M_vectors[index]
        H_anis = magnetosome.anisotropy_field()
        magnetosome.M = original_M # Restore original M

        H_dip = self.calculate_dipolar_field(index, current_M_vectors)
        H_th = self.calculate_thermal_field(magnetosome) # Calculate thermal field

        # Ignored fields: H_demag, H_exch
        H_eff = self.H_ext + H_anis + H_dip + H_th # Add thermal field
        return H_eff

    def llg_equation(self, t, M_flat):
        """
        LLG equation including the thermal field calculated in calculate_Heff.
        (Structure is the same, but Heff now includes Hth)
        """
        dM_dt_flat = np.zeros_like(M_flat)
        num_magnetosomes = len(self.magnetosomes)
        current_M_vectors = {i: M_flat[i*3:(i+1)*3] for i in range(num_magnetosomes)}

        for i in range(num_magnetosomes):
            M = current_M_vectors[i]
            Ms_i = self.magnetosomes[i].Ms
            if Ms_i < 1e-9 or np.linalg.norm(M) < 1e-9:
                 dM_dt_flat[i*3:(i+1)*3] = np.zeros(3)
                 continue

            # Heff now implicitly includes H_th because calculate_Heff calls calculate_thermal_field
            H_eff = self.calculate_Heff(i, current_M_vectors)

            prefactor1 = -GAMMA_LL / (1 + ALPHA**2)
            prefactor2 = prefactor1 * ALPHA / Ms_i
            M_cross_Heff = np.cross(M, H_eff)
            M_cross_M_cross_Heff = np.cross(M, M_cross_Heff)
            dM_dt = prefactor1 * M_cross_Heff + prefactor2 * M_cross_M_cross_Heff
            dM_dt_flat[i*3:(i+1)*3] = dM_dt
        return dM_dt_flat

    def run(self, t_span, dt_max, t_eval=None):
        """ Runs the simulation (pass dt_max to __init__ for thermal field scaling) """
        # Store dt_max for thermal field calculation approximation
        self.approx_dt = dt_max

        initial_M_flat = np.concatenate([m.M for m in self.magnetosomes])
        sol = solve_ivp(
            fun=self.llg_equation, t_span=t_span, y0=initial_M_flat,
            method='RK45', # Note: RK45 is not ideal for SDEs
            max_step=dt_max, t_eval=t_eval,
            # dense_output=True # May need dense output for finer time resolution if t_eval is coarse
        )
        if sol.status == 0:
            final_M_flat = sol.y[:, -1]
            for i, m in enumerate(self.magnetosomes):
                m.M = final_M_flat[i*3:(i+1)*3]
                m.normalize_M()
        else: print(f"Warning: ODE solver status {sol.status}: {sol.message}")
        return sol

    # Add the plot_state method here as well if needed
    # (Code omitted for brevity)
    def plot_state(self, sol=None, time_index=-1, plot_trajectory_indices=None):
        """ Visualizes the state (Copied from magnetosome_sim_env) """
        # ... (same code as before) ...
        pass # Placeholder - include the full plot_state code here

# <<< END of pasted/modified code >>>


# ==========================================================
# --- Thermal Stability Simulation Setup ---
# ==========================================================

# --- Parameters ---
# Use a single magnetosome for simplicity
magnetosome_diameter = 45e-9
magnetosome_radius = magnetosome_diameter / 2
magnetosome_volume = (4/3) * np.pi * magnetosome_radius**3
# Anisotropy constant - stability depends strongly on Ku*V
magnetosome_Ku = 1.1e4 # J/m^3 (Relatively low barrier for demonstration)
# magnetosome_Ku = 2.0e4 # J/m^3 (Higher barrier)
easy_axis_direction = [0, 0, 1] # Easy axis along +z
simulation_temperature = 300 # Kelvin

# --- Calculate Energy Barrier ---
Delta_E = magnetosome_Ku * magnetosome_volume
Thermal_Energy = KB * simulation_temperature
Stability_Ratio = Delta_E / Thermal_Energy
print(f"Energy Barrier (Ku*V): {Delta_E:.2e} J")
print(f"Thermal Energy (kB*T): {Thermal_Energy:.2e} J")
print(f"Stability Ratio (Ku*V / kB*T): {Stability_Ratio:.2f}")
# If Ratio >> 1 (e.g., > 40-60), the state should be relatively stable.
# If Ratio is low, expect frequent thermal switching.

# --- Create the Magnetosome ---
# Start aligned perfectly with the easy axis
initial_M = easy_axis_direction
m1 = Magnetosome(position=[0, 0, 0],
                 initial_M_direction=initial_M,
                 volume=magnetosome_volume,
                 Ku=magnetosome_Ku,
                 easy_axis=easy_axis_direction)

magnetosomes_list = [m1]

# --- External Field ---
H_ext_vec = [0, 0, 0] # No external field

# --- Create and Run Simulation ---
# Simulation time parameters - need long time to observe potential flips
t_start = 0
t_end = 20e-9      # Simulate for 20 nanoseconds (adjust based on stability ratio)
dt_max = 1e-13     # Use a smaller max time step for thermal simulation
num_time_points = 1001 # Number of points for detailed output
t_eval_points = np.linspace(t_start, t_end, num_time_points)

# Pass temperature and dt_max to the Simulation constructor
simulation = Simulation(magnetosomes=magnetosomes_list,
                        H_ext=H_ext_vec,
                        temperature=simulation_temperature,
                        approx_dt=dt_max) # Pass dt_max for thermal field scaling

print(f"Starting thermal stability simulation at T={simulation_temperature} K...")
# Run multiple times to see stochastic behavior
num_runs = 3
plt.figure(figsize=(10, 6))

for run in range(num_runs):
    print(f"  Run {run+1}/{num_runs}...")
    # Reset initial state for each run (important!)
    m1.M = (np.array(initial_M) / np.linalg.norm(initial_M)) * MS
    solution = simulation.run(t_span=[t_start, t_end], dt_max=dt_max, t_eval=t_eval_points)

    # --- Analyze and Plot Mz(t) ---
    times = solution.t
    mz_values = solution.y[2, :] / MS # Mz is the 3rd component (index 2)

    plt.plot(times * 1e9, mz_values, label=f'Run {run+1}')

print("Simulations finished.")

plt.xlabel('Time (ns)')
plt.ylabel('Normalized Mz component (Mz/Ms)')
plt.title(f'Thermal Stability Simulation (T={simulation_temperature} K, Ku*V/kBT={Stability_Ratio:.1f})')
plt.ylim(-1.1, 1.1)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

Energy Barrier (Ku*V): 5.25e-19 J
Thermal Energy (kB*T): 4.14e-21 J
Stability Ratio (Ku*V / kB*T): 126.71
Starting thermal stability simulation at T=300 K...
  Run 1/3...
